# Now, every face will be cropped and saved

In [1]:
!pwd

/home/xilinx/jupyter_notebooks/Final_present


In [1]:
# In this cell, some libraries were imported
import cv2
import sys
import os
from PIL import Image, ImageDraw
import pylab
import time

In [ ]:

# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)
 
# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open("Xilinx_Hackathon_2017").sheet1


###### ORB
# imports for ORB
import numpy as np
import cv2

# Initiate ORB detector
orb = cv2.ORB_create()

# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

import string
import re
####################################
######   Configurable items:  ######
####################################
# arbitary number. Fine tune once you get here
thres_hold = 250


In [2]:
# Face Detection Function
def detectFaces(image_name):
    print ("Face Detection Start.")
    # Read the image and convert to gray to reduce the data
    img = cv2.imread(image_name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)#Color => Gray
    
    # The haarcascades classifier is used to train data
    #face_cascade = cv2.CascadeClassifier("/usr/share/opencv/haarcascades/haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)#1.3 and 5are the min and max windows of the treatures
    result = []
    for (x,y,width,height) in faces:
        result.append((x,y,x+width,y+height))
    print ("Face Detection Complete.")
    return result

In [3]:
#Crop faces and save them in the same directory
filepath ="/home/xilinx/jupyter_notebooks/Final_presents/images/"
dir_path ="/home/xilinx/jupyter_notebooks/Final_presents/"
filecount = len(os.listdir(filepath))-1
image_count = 1#count is the number of images
face_cascade = cv2.CascadeClassifier("/usr/share/opencv/haarcascades/haarcascade_frontalface_default.xml")
for fn in os.listdir(filepath): #fn 表示的是文件名
    start = time.time()
    if image_count <= filecount:
        image_name = str(image_count) + '.JPG'
        image_path = filepath + image_name
        image_new = dir_path + image_name
        #print (image_path)
        #print (image_new)
        os.system('cp '+(image_path)+ (' /home/xilinx/jupyter_notebooks/Final_presents/'))
        faces = detectFaces(image_name)
        if not faces:
            print ("Error to detect face")
        if faces:
            #All croped face images will be saved in a subdirectory
            face_name = image_name.split('.')[0]
            #os.mkdir(save_dir)
            count = 0
            for (x1,y1,x2,y2) in faces:
                file_name = os.path.join(dir_path,face_name+str(count)+".jpg")
                Image.open(image_name).crop((x1,y1,x2,y2)).save(file_name)
                #os.system('rm -rf '+(image_path)+' /home/xilinx/jupyter_notebooks/OpenCV/Face_Detection/')
                count+=1 
            
            
            
            
            # Integration START
            
            
            
            # New face from the webcam
            image = file_name

            # Find the Key point
            img = cv2.imread(image,0)          # queryImage

            # find the keypoints and descriptors with ORB
            kp1, des1 = orb.detectAndCompute(img,None)
            
            # Extract and print all of the values
            all_faces_str = sheet.get_all_values()
            
            
            # convert google doc strings to numbers
            data = all_faces_str[1:]
            face_order = 0
            for row in data:

                face_order += 1
                myTotal = np.array([])
                for element in row:
                    myArray = np.array(eval(element))
                    myTotal = np.vstack([myTotal,myArray]) if myTotal.size else myArray


                # convert data type
                myTotal_uint8 = np.uint8(myTotal) 

                # Add each image descriptor list from the database
                clusters = np.array([myTotal_uint8])
                bf.add(clusters)

                # Train: Does nothing for BruteForceMatcher though.
                bf.train()

                matches = bf.match(des1,myTotal_uint8)
                matches = sorted(matches, key = lambda x:x.distance)

                facest_face = 0
                numb_matches = (len(matches))
                if numb_matches > facest_face:
                    facest_face = numb_matches

                face_found = False
                if facest_face > thres_hold:
                    face_found = True
                    found_face_order = face_order
            
            
            
            # Add face to database if not found, otherwise report the face
            if face_found == True:
                print ("FACE FOUND! Face number:")
                print (found_face_order)
            else:
                print ("Face NOT found. adding face to database")
                sheet.append_row('')
                col_in = 1
                row_in = sheet.row_count

                collumn_list = []
                des_cnt = 0
                des_per_cell = 100
                for collumn in des1:

                    collumn_list.append(collumn.tolist())
                    # Append des_per_cell # of collumns into a single collumn and upload
                    if des_cnt >= des_per_cell -1:
                        print ("cloud")
                        sheet.update_cell(row_in, col_in, collumn_list)
                        collumn_list = []
                        col_in += 1
                        des_cnt = 0
                    else:
                        des_cnt += 1

            
            
            # Integration END
            
            
            
            os.system('rm -rf '+(image_new))
            print("The " + str(image_count) +" image were done.")
            print("Congratulation! The total of the " + str(count) + " faces in the " +str(image_count) + " image.")
    end = time.time()
    TimeSpan = end - start
    if image_count <= filecount:
        print ("The time of " + str(image_count) + " image is " +str(TimeSpan) + " s.")
    image_count = image_count + 1   

Face Detection Start.
Face Detection Complete.
The 1 image were done.


NameError: name 'count' is not defined

In [ ]:
# Initiate ORB detector
orb = cv2.ORB_create()
img1 = cv2.imread('20.jpg',cv2.COLOR_BGR2GRAY)
img2 = cv2.imread('31.jpg',cv2.COLOR_BGR2GRAY)

In [ ]:
# find the keypoints and descriptors with SIFT for face 1
kp1, des1 = orb.detectAndCompute(img1,None)
imgOut1 = cv2.drawKeypoints(img1,kp,None, color=(255,0,0), flags=0)
plt.imshow(imgOut1),plt.show()

# find the keypoints and descriptors with SIFT for face 2
kp2, des2 = orb.detectAndCompute(img2,None)
imgOut2 = cv2.drawKeypoints(img2,kp,None, color=(255,0,0), flags=0)
plt.imshow(imgOut2),plt.show()

In [ ]:
# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors.
matches = bf.match(des1,des2)

# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)

In [ ]:
def drawMatches(img1, kp1, img2, kp2, matches):
    """
    My own implementation of cv2.drawMatches as OpenCV 2.4.9
    does not have this function available but it's supported in
    OpenCV 3.0.0

    This function takes in two images with their associated 
    keypoints, as well as a list of DMatch data structure (matches) 
    that contains which keypoints matched in which images.

    An image will be produced where a montage is shown with
    the first image followed by the second image beside it.

    Keypoints are delineated with circles, while lines are connected
    between matching keypoints.

    img1,img2 - Grayscale images
    kp1,kp2 - Detected list of keypoints through any of the OpenCV keypoint 
              detection algorithms
    matches - A list of matches of corresponding keypoints through any
              OpenCV keypoint matching algorithm
    """

    # Create a new output image that concatenates the two images together
    # (a.k.a) a montage
    rows1 = img1.shape[0]
    cols1 = img1.shape[1]
    rows2 = img2.shape[0]
    cols2 = img2.shape[1]

    out = np.zeros((max([rows1,rows2]),cols1+cols2,3), dtype='uint8')

    # Place the first image to the left
    out[:rows1,:cols1] = np.dstack([img1, img1, img1])

    # Place the next image to the right of it
    out[:rows2,cols1:] = np.dstack([img2, img2, img2])

    # For each pair of points we have between both images
    # draw circles, then connect a line between them
    for mat in matches:

        # Get the matching keypoints for each of the images
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx

        # x - columns
        # y - rows
        (x1,y1) = kp1[img1_idx].pt
        (x2,y2) = kp2[img2_idx].pt

        # Draw a small circle at both co-ordinates
        # radius 4
        # colour blue
        # thickness = 1
        cv2.circle(out, (int(x1),int(y1)), 4, (255, 0, 0), 1)   
        cv2.circle(out, (int(x2)+cols1,int(y2)), 4, (255, 0, 0), 1)

        # Draw a line in between the two points
        # thickness = 1
        # colour blue
        cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (255, 0, 0), 1)

    return out

In [ ]:
# Draw first 10 matches.
print (len(matches))
img3 = drawMatches(img1,kp1,img2,kp2,matches[:10])

plt.imshow(img3),plt.show()